In [1]:
import sys
import math
from collections import defaultdict
import itertools

### 3. Unsmoothed n-grams

To start, you will write a program that computes unsmoothed unigram and bigram probabilities from  the training corpus. 
You are given a tokenized opinion spam corpus as input (see Section 2). You may want to do additional preprocessing, based on the design decisions you make 1. Make sure to explain preprocessing decisions you make clearly in the report. Note that you may use existing tools just for the purpose of preprocessing  but you must write the code for gathering n-gram counts and computing n-gram probabilities yourself. 
For example, consider the simple corpus consisting of the sole sentence:
the students like the assignment

Part of what your program would compute for a unigram and bigram model, for example, would be the
following:

P(the) = 0.4, P (like) = 0.2 

P (the|like) = 1.0, P (students|the) = 0.5

Preprocessing: The files included are already tokenized and hence it should be straightforward to 
obtain the tokens by using space as the delimiter. Feel free to do any other preprocessing that you might  think is important for this corpus. Do not forget to describe and explain your pre-processing choices in your report.

In [2]:
with open('train.txt', 'r') as f:
    # Read the entire file content
    train_text = f.readlines()

In [3]:
train_text

['I booked two rooms four months in advance at the Talbott . We were placed on the top floor next to the elevators , which are used all night long . When speaking to the front desk , I was told that they were simply honoring my request for an upper floor , which I had requested for a better view . I am looking at a brick wall , and getting no sleep . He also told me that they had received complaints before from guests on the 16th floor , and were aware of the noise problem . Why then did they place us on this floor when the hotel is not totally booked ? A request for an upper floor does not constitute placing someone on the TOP floor and using that request to justify this . If you decide to stay here , request a room on a lower floor and away from the elevator ! I spoke at length when booking my two rooms about my preferences . This is simply poor treatment of a guest whom they believed would not complain .\n',
 "I LOVED this hotel . The room was so chic and trendy , the bed was comfor

In [4]:
# tokenize the train text
def tokenize(text):
    train_tokens = [sentence.split() for sentence in text]
    # print(train_tokens)
    print(f"Number of train tokens: {len(set(itertools.chain(*train_tokens)))}")

    # convert all tokens to lowercase. this will treat the same words in different case as one word and this will help the frequency
    train_tokens_lc = []
    for sentence in train_tokens:
        train_tokens_lc.append([token.lower() for token in sentence])

    print(f"Number of lowercased train tokens: {len(set(itertools.chain(*train_tokens_lc)))}")

    return train_tokens_lc

train_tokens = tokenize(train_text)

Number of train tokens: 7256
Number of lowercased train tokens: 6380


In [5]:
# Get the unigram and bigram counts
def get_unigram_bigram_count(wordlist):
    unigram_counts = defaultdict(int)
    bigram_counts = defaultdict(int)
    for sentence in wordlist:
        for i in range(len(sentence) - 1):
            unigram_counts[sentence[i]] += 1
            bigram_counts[(sentence[i], sentence[i+1])] += 1

        unigram_counts[sentence[-1]] += 1 # Count the last word

    return unigram_counts, bigram_counts


uni_c, bi_c = get_unigram_bigram_count(train_tokens)

display(uni_c, bi_c)

defaultdict(int,
            {'i': 1706,
             'booked': 86,
             'two': 128,
             'rooms': 201,
             'four': 20,
             'months': 8,
             'in': 1259,
             'advance': 7,
             'at': 745,
             'the': 5292,
             'talbott': 28,
             '.': 4692,
             'we': 1116,
             'were': 578,
             'placed': 8,
             'on': 640,
             'top': 40,
             'floor': 137,
             'next': 107,
             'to': 2090,
             'elevators': 32,
             ',': 2949,
             'which': 176,
             'are': 310,
             'used': 35,
             'all': 278,
             'night': 199,
             'long': 39,
             'when': 273,
             'speaking': 3,
             'front': 125,
             'desk': 159,
             'was': 1826,
             'told': 77,
             'that': 694,
             'they': 489,
             'simply': 19,
             'honoring': 1,

defaultdict(int,
            {('i', 'booked'): 21,
             ('booked', 'two'): 1,
             ('two', 'rooms'): 3,
             ('rooms', 'four'): 1,
             ('four', 'months'): 1,
             ('months', 'in'): 2,
             ('in', 'advance'): 7,
             ('advance', 'at'): 1,
             ('at', 'the'): 332,
             ('the', 'talbott'): 26,
             ('talbott', '.'): 9,
             ('.', 'we'): 345,
             ('we', 'were'): 177,
             ('were', 'placed'): 1,
             ('placed', 'on'): 2,
             ('on', 'the'): 228,
             ('the', 'top'): 12,
             ('top', 'floor'): 5,
             ('floor', 'next'): 1,
             ('next', 'to'): 25,
             ('to', 'the'): 264,
             ('the', 'elevators'): 11,
             ('elevators', ','): 6,
             (',', 'which'): 53,
             ('which', 'are'): 3,
             ('are', 'used'): 1,
             ('used', 'all'): 1,
             ('all', 'night'): 13,
             ('night',

In [6]:
def compute_unigram_prob(unigram_counts):
    # Compute unigram probabilities.
    total_tokens = sum(unigram_counts.values())
    unigram_prob = {}
    for token, count in unigram_counts.items():
        unigram_prob[token] = count / total_tokens
    return unigram_prob

def compute_bigram_prob(bigram_counts, unigram_counts):
    # Compute bigram probabilities.
    bigram_prob = {}
    for bigram, count in bigram_counts.items():
        prev_token = bigram[0]
        bigram_prob[bigram] = count / unigram_counts[prev_token]
    return bigram_prob

unigram_probs = compute_unigram_prob(uni_c)
bigram_probs = compute_bigram_prob(bi_c, uni_c)

display(unigram_probs, bigram_probs)


{'i': 0.019022345122875876,
 'booked': 0.0009589224387850676,
 'two': 0.001427233397261496,
 'rooms': 0.0022412024441371926,
 'four': 0.00022300521832210873,
 'months': 8.92020873288435e-05,
 'in': 0.014038178493376746,
 'advance': 7.805182641273806e-05,
 'at': 0.00830694438249855,
 'the': 0.05900718076802997,
 'talbott': 0.00031220730565095225,
 '.': 0.05231702421836671,
 'we': 0.012443691182373668,
 'were': 0.006444850809508943,
 'placed': 8.92020873288435e-05,
 'on': 0.0071361669863074795,
 'top': 0.00044601043664421747,
 'floor': 0.001527585745506445,
 'next': 0.0011930779180232818,
 'to': 0.023304045314660363,
 'elevators': 0.000356808349315374,
 ',': 0.032882119441594936,
 'which': 0.0019624459212345567,
 'are': 0.0034565808839926854,
 'used': 0.0003902591320636903,
 'all': 0.0030997725346773113,
 'night': 0.002218901922304982,
 'long': 0.00043486017572811204,
 'when': 0.003044021230096784,
 'speaking': 3.345078274831631e-05,
 'front': 0.0013937826145131796,
 'desk': 0.0017728914

{('i', 'booked'): 0.0123094958968347,
 ('booked', 'two'): 0.011627906976744186,
 ('two', 'rooms'): 0.0234375,
 ('rooms', 'four'): 0.004975124378109453,
 ('four', 'months'): 0.05,
 ('months', 'in'): 0.25,
 ('in', 'advance'): 0.005559968228752979,
 ('advance', 'at'): 0.14285714285714285,
 ('at', 'the'): 0.44563758389261743,
 ('the', 'talbott'): 0.0049130763416477706,
 ('talbott', '.'): 0.32142857142857145,
 ('.', 'we'): 0.07352941176470588,
 ('we', 'were'): 0.1586021505376344,
 ('were', 'placed'): 0.0017301038062283738,
 ('placed', 'on'): 0.25,
 ('on', 'the'): 0.35625,
 ('the', 'top'): 0.0022675736961451248,
 ('top', 'floor'): 0.125,
 ('floor', 'next'): 0.0072992700729927005,
 ('next', 'to'): 0.2336448598130841,
 ('to', 'the'): 0.12631578947368421,
 ('the', 'elevators'): 0.0020786092214663643,
 ('elevators', ','): 0.1875,
 (',', 'which'): 0.01797219396405561,
 ('which', 'are'): 0.017045454545454544,
 ('are', 'used'): 0.0032258064516129032,
 ('used', 'all'): 0.02857142857142857,
 ('all', 

Testing the sample sentence:

"the students like the assignment"

In [7]:
sample_tokens = tokenize(["the students like the assignment"])
sample_uni_c, sample_bi_c = get_unigram_bigram_count(sample_tokens)

display(sample_uni_c, sample_bi_c)

display(compute_unigram_prob(sample_uni_c))
display(compute_bigram_prob(sample_bi_c, sample_uni_c))

Number of train tokens: 4
Number of lowercased train tokens: 4


defaultdict(int, {'the': 2, 'students': 1, 'like': 1, 'assignment': 1})

defaultdict(int,
            {('the', 'students'): 1,
             ('students', 'like'): 1,
             ('like', 'the'): 1,
             ('the', 'assignment'): 1})

{'the': 0.4, 'students': 0.2, 'like': 0.2, 'assignment': 0.2}

{('the', 'students'): 0.5,
 ('students', 'like'): 1.0,
 ('like', 'the'): 1.0,
 ('the', 'assignment'): 0.5}

### 4. Smoothing and unknown words

Firstly, you should implement one or more than one methods to handle unknown words. Then You will need to implement two smoothing methods (e.g. Laplace, Add-k smoothing with different k). Teams can choose any method(s) that they want for each. The report should make clear what methods were selected, providing a description for any non-standard approach.

In [8]:
def replace_with_unknown(tokens, word_counts, thresh=2, train_set=True):
    # we use the no prior vocab replacement solution 
    # where we replace words with a frequency less than the threshold with the <UNK> token.
    updated_tokens = []
    for sentence in tokens:
        updated_sentence = []
        for word in sentence:
            if train_set:
                if word_counts[word] >= thresh:
                    updated_sentence.append(word)
                else:
                    updated_sentence.append("<UNK>")
            else:
                if word in word_counts and word_counts[word] >= thresh:
                    updated_sentence.append(word)
                else:
                    updated_sentence.append("<UNK>")

        updated_tokens.append(updated_sentence)

    return updated_tokens

train_tokens = replace_with_unknown(train_tokens, uni_c, thresh=2, train_set=True)

print(f"Number of train tokens after replacing with <UNK>: {len(set(itertools.chain(*train_tokens)))}")

Number of train tokens after replacing with <UNK>: 3116


Add-K smoothing formulation:

$P_{\text{Add-k}}(x_i \mid x_{i-1}) = \frac{c(x_{i-1}, x_i) + k}{c(x_{i-1}) + kV}$


In [9]:
def addk_unigram_prob(unigram_counts, k=1):
    unigram_prob_addk = {}
    total_tokens = sum(unigram_counts.values())
    vocabulary_size = len(unigram_counts)

    for token, count in unigram_counts.items():
        unigram_prob_addk[token] = (count + k) / (total_tokens + k * vocabulary_size)
    
    return unigram_prob_addk


def addk_bigram_prob(bigram_counts, unigram_counts, k=1):
    # add-k smoothing adds K to all of the counts before normalizing into probabilities
    # add-1 smoothing for k=1 is called laplace smoothing
    smooth_bigram_prob = {}
    vocabulary_size = len(unigram_counts)

    for bigram, count in bigram_counts.items():
        prev_token = bigram[0]
        smooth_bigram_prob[bigram] = (count + k) / (unigram_counts[prev_token] + k * vocabulary_size)

    # Handle bigrams not seen in training (zero counts)
    # for prev_token in unigram_counts:
    #     for token in unigram_counts:
    #         bigram = (prev_token, token)
    #         if bigram not in smooth_bigram_prob:
    #             smooth_bigram_prob[bigram] = k / (unigram_counts[prev_token] + k * vocabulary_size)

    return smooth_bigram_prob

smooth_unigram_probs = addk_unigram_prob(uni_c, k=1)
smooth_bigram_probs = addk_bigram_prob(bi_c, uni_c, k=1)

display(smooth_unigram_probs, smooth_bigram_probs)

{'i': 0.017769403730846102,
 'booked': 0.0009056462358427715,
 'two': 0.001342854763491006,
 'rooms': 0.0021027648234510326,
 'four': 0.00021860426382411727,
 'months': 9.368754163890739e-05,
 'in': 0.013116255829447036,
 'advance': 8.327781479013991e-05,
 'at': 0.007765656229180546,
 'the': 0.055098684210526314,
 'talbott': 0.00030188207861425715,
 '.': 0.04885284810126582,
 'we': 0.011627664890073285,
 'were': 0.0060272318454363755,
 'placed': 9.368754163890739e-05,
 'on': 0.00667263491005996,
 'top': 0.00042679880079946704,
 'floor': 0.0014365423051299134,
 'next': 0.0011242504996668888,
 'to': 0.021766738840772817,
 'elevators': 0.00034352098600932713,
 ',': 0.03070869420386409,
 'which': 0.0018425216522318455,
 'are': 0.0032374250499666887,
 'used': 0.00037475016655562956,
 'all': 0.0029043137908061293,
 'night': 0.0020819453697534978,
 'long': 0.00041638907395069954,
 'when': 0.002852265156562292,
 'speaking': 4.1638907395069954e-05,
 'front': 0.0013116255829447034,
 'desk': 0.00

{('i', 'booked'): 0.002720751916893396,
 ('booked', 'two'): 0.0003093102381688834,
 ('two', 'rooms'): 0.0006146281499692685,
 ('rooms', 'four'): 0.000303905181583346,
 ('four', 'months'): 0.0003125,
 ('months', 'in'): 0.000469630557294928,
 ('in', 'advance'): 0.0010472574944364445,
 ('advance', 'at'): 0.0003131360576170346,
 ('at', 'the'): 0.04673684210526316,
 ('the', 'talbott'): 0.002313228238519534,
 ('talbott', '.'): 0.001560549313358302,
 ('.', 'we'): 0.03125,
 ('we', 'were'): 0.023745997865528282,
 ('were', 'placed'): 0.0002874389192296637,
 ('placed', 'on'): 0.000469630557294928,
 ('on', 'the'): 0.03262108262108262,
 ('the', 'top'): 0.001113776559287183,
 ('top', 'floor'): 0.0009345794392523365,
 ('floor', 'next'): 0.0003068896731624981,
 ('next', 'to'): 0.004008016032064128,
 ('to', 'the'): 0.03128689492325856,
 ('the', 'elevators'): 0.001028101439342015,
 ('elevators', ','): 0.001091703056768559,
 (',', 'which'): 0.005788401757959053,
 ('which', 'are'): 0.0006101281269066504,


Linear Interpolation smoothing formulation:

$P_{\lambda}(w \mid u, v) = \lambda_3 P_{\text{MLE}}(w \mid u, v) + \lambda_2 P_{\text{MLE}}(w \mid v) + \lambda_1 P_{\text{MLE}}(w)$

where, $\sum_{i} \lambda_i = 1$


In [10]:
def linear_intrpl_bigram_prob(unigram_prob, bigram_prob, lambda1, lambda2):
    # following the above formulation
    smooth_bigram_prob = {}
    for bigram, bi_p in bigram_prob.items():
        uni_p = unigram_prob.get(bigram[1], 0)
        smooth_bigram_prob[bigram] = lambda1 * bi_p + lambda2 * uni_p

    return smooth_bigram_prob

smooth_bigram_probs = linear_intrpl_bigram_prob(unigram_probs, bigram_probs, lambda1=0.3, lambda2=0.7)

display(smooth_bigram_probs)

{('i', 'booked'): 0.004364094476199957,
 ('booked', 'two'): 0.004487435471106303,
 ('two', 'rooms'): 0.008600091710896034,
 ('rooms', 'four'): 0.001648640966258312,
 ('four', 'months'): 0.01506244146113019,
 ('months', 'in'): 0.08482672494536372,
 ('in', 'advance'): 0.0017226267471148102,
 ('advance', 'at'): 0.048672003924891835,
 ('at', 'the'): 0.1749963017054062,
 ('the', 'talbott'): 0.0016924680164499978,
 ('talbott', '.'): 0.13305048838142813,
 ('.', 'we'): 0.030769407357073335,
 ('we', 'were'): 0.05209204072794658,
 ('were', 'placed'): 0.0005814726029987026,
 ('placed', 'on'): 0.07999531689041524,
 ('on', 'the'): 0.14818002653762097,
 ('the', 'top'): 0.0009924794144944896,
 ('top', 'floor'): 0.03856931002185451,
 ('floor', 'next'): 0.003024935564514107,
 ('next', 'to'): 0.08640628966418748,
 ('to', 'the'): 0.07919976337972624,
 ('the', 'elevators'): 0.0008733486109606711,
 ('elevators', ','): 0.07926748360911645,
 (',', 'which'): 0.006765370334080872,
 ('which', 'are'): 0.00753324

### 5. Perplexity on Validation Set

Implement code to compute the perplexity of a “development set.” (“development set” is just another way to refer to the validation set — part of a dataset that is distinct from the training portion.) Compute and report the perplexity of your model (with variations) on it. Compute perplexity as follows:

$PP = \left( \prod_{i=1}^{N} \frac{1}{P(w_i \mid w_{i-1}, \dots, w_{i-n+1})} \right)^{1/N}$

$PP = \exp \left( \frac{1}{N} \sum_{i=1}^{N} -\log P(w_i \mid w_{i-1}, \dots, w_{i-n+1}) \right)$

where N is the total number of tokens in the test corpus and P(w_i \mid w_{i-1}, \dots, w_{i-n+1}) \right) is the n-gram 
probability of your model. Under the second definition above, perplexity is a function of the average (per-word) log probability: use this to avoid numerical computation errors.

If you experimented with more than one type of smoothing and unknown word handling, you should report and compare the perplexity results of experiments among some of them.

In [11]:
def compute_perplexity(dev_sentences, bigram_prob, unigram_prob, n=2):
    """
    Compute the perplexity of the model on the development set.

    Parameters:
    - development_sentences: List of preprocessed development sentences.
    - bigram_prob_model: Dictionary of bigram probabilities.
    - unigram_prob_model: Dictionary of unigram probabilities (used for linear interpolation if applicable).
    - n: The order of n-grams (default is 2 for bigram).

    Returns:
    - perplexity: The computed perplexity value.
    """
    N = 0
    log_probability_sum = 0.0

    for sentence in dev_sentences:
        for i in range(len(sentence) - 1):
            N += 1
            prev_token = sentence[i]
            current_token = sentence[i+1]
            bigram = (prev_token, current_token)
            prob = bigram_prob.get(bigram, 0)

            # prob = bigram_probs.get(w1, {}).get(w2, 0)
            # if prob == 0:
            #     # Can't compute perplexity, since log(0) is undefined
            #     return float('inf')
            # log_prob += -math.log(prob)            

            # if prob == 0.0: # too small to store
            #     log_probability_sum += sys.float_info.min # smallest float
            # else:
            #     log_probability_sum += math.log(prob)


            if prob > 0:
                log_probability_sum += math.log(prob)
            else:
                # If probability is zero (which shouldn't happen with smoothing), assign a large negative value
                print("HOHOHO")
                log_probability_sum += -math.inf

    print(N)

    # Compute perplexity
    if N == 0:
        return float('inf')  # Avoid division by zero
    avg_log_prob = log_probability_sum / N
    perplexity = math.exp(-avg_log_prob)

    return perplexity

### Putting it all together

In [12]:
with open('train.txt', 'r') as f:
    # Read the entire file content
    train_text = f.readlines()

train_tokens = tokenize(train_text)

uni_c, bi_c = get_unigram_bigram_count(train_tokens)

train_tokens_w_unk = replace_with_unknown(train_tokens, uni_c, thresh=2, train_set=True)

uni_c_w_unk, bi_c_w_unk = get_unigram_bigram_count(train_tokens_w_unk)

laplace_unigram_probs = addk_unigram_prob(uni_c_w_unk, k=1)
laplace_bigram_probs = addk_bigram_prob(bi_c_w_unk, uni_c_w_unk, k=1)

addk_unigram_probs = addk_unigram_prob(uni_c_w_unk, k=0.5)
addk_bigram_probs = addk_bigram_prob(bi_c_w_unk, uni_c_w_unk, k=0.5)

unigram_probs_w_unk = compute_unigram_prob(uni_c_w_unk)
bigram_probs_w_unk = compute_bigram_prob(bi_c_w_unk, uni_c_w_unk)
linear_intrpl_bigram_probs = linear_intrpl_bigram_prob(unigram_probs_w_unk, bigram_probs_w_unk, lambda1=0.3, lambda2=0.7)

Number of train tokens: 7256
Number of lowercased train tokens: 6380


In [13]:
with open('train.txt', 'r') as f:
    # Read the entire file content
    val_text = f.readlines()

val_tokens = tokenize(val_text)

val_tokens_w_unk = replace_with_unknown(val_tokens, uni_c_w_unk, thresh=2, train_set=False)

preplexity = compute_perplexity(val_tokens_w_unk, laplace_bigram_probs, laplace_unigram_probs, n=2)

print(f"Perplexity with laplace smoothing: {preplexity}")

preplexity = compute_perplexity(val_tokens_w_unk, addk_bigram_probs, addk_unigram_probs, n=2)

print(f"Perplexity with add-{0.5} smoothing: {preplexity}")

preplexity = compute_perplexity(val_tokens_w_unk, linear_intrpl_bigram_probs, addk_unigram_probs, n=2)

print(f"Perplexity with linear interpolation smoothing: {preplexity}")

Number of train tokens: 7256
Number of lowercased train tokens: 6380
89172
Perplexity with laplace smoothing: 387.45876428298
89172
Perplexity with add-0.5 smoothing: 258.5204821985406
89172
Perplexity with linear interpolation smoothing: 73.21132385314
